# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import KNNImputer
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
data=pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv')
data.head(10)

# Data Preprocessing

In [ ]:
data[data.columns[data.isna().sum() > 0]].isna().mean()*100

In [ ]:
data.drop(['Alley','FireplaceQu','PoolQC','Fence','MiscFeature','Id'], inplace=True, axis=1)
data[data.columns[data.isna().sum() > 0]].isna().mean()*100

In [ ]:
X=data.drop('SalePrice', axis=1)
y=data['SalePrice']

In [ ]:
x_en=pd.get_dummies(X,drop_first=True)
imputer=KNNImputer()
imputer.fit(x_en)
x=imputer.transform(x_en)

In [ ]:
x=pd.DataFrame(x, columns=x_en.columns)
x

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
lof = LocalOutlierFactor()
yhat = lof.fit_predict(x.to_numpy())
mask=yhat!=-1

In [ ]:
x_train=x.to_numpy()[mask, :]
x_train=pd.DataFrame(x_train, columns=x.columns)
y_train=y[mask]
print(x_train.shape, y_train.shape)

In [ ]:
mm_scaler=MinMaxScaler()
x_scaled=pd.DataFrame(mm_scaler.fit_transform(x_train), columns=x_train.columns)
x_scaled.head()

In [ ]:
target_scaler=MinMaxScaler()
y_data=pd.DataFrame(y_train)
target_scaler.fit(y_data)
y_scaled=target_scaler.transform(y_data)
y_scaled

In [ ]:
dtr=DecisionTreeRegressor()
dtr.fit(x_scaled,y_scaled)
rfe=RFE(dtr,15)
rfe=rfe.fit(x_scaled,y_scaled)

In [ ]:
x_scaled.columns[rfe.support_]

In [ ]:
selected_list=['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'GrLivArea', 'GarageCars', 'GarageArea', 'MoSold', 'CentralAir_Y']
x_selected=x_scaled[selected_list]
x_selected.head()

In [ ]:
x_features=x_selected.columns
x_features

# Make Model

In [ ]:
final_gradient=GradientBoostingRegressor(learning_rate= 0.10903, max_depth= 5, n_estimators= 54)
final_gradient.fit(x_selected, y_scaled)

In [ ]:
test=pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv')
test.head()

In [ ]:
Id_pred=test['Id']
test.drop(['Alley','FireplaceQu','PoolQC','Fence','MiscFeature','Id'], inplace=True, axis=1)
test_en=pd.get_dummies(test,drop_first=True)
test_en.shape

In [ ]:
result_test= x_en.append(test_en, sort=False)
result_test.shape

In [ ]:
test_en_2=result_test[1460:2919]
test_en_2.shape

In [ ]:
test_en_2[test_en_2.columns[test_en_2.isna().sum() > 0]].isna().mean()*100

In [ ]:
test_en_3=imputer.transform(test_en_2)
test_en_3=pd.DataFrame(test_en_3, columns=test_en_2.columns)
test_en_3[test_en_3.columns[test_en_3.isna().sum() > 0]].isna().mean()*100

In [ ]:
test_scaled = pd.DataFrame(mm_scaler.transform(test_en_3), columns=test_en_3.columns)
test_scaled.head()

In [ ]:
test_selected=test_scaled[x_features]
test_selected.head()

# Prediction

In [ ]:
ypred_scale=final_gradient.predict(test_selected)
ypred_scale=pd.DataFrame(ypred_scale)
ypred=target_scaler.inverse_transform(ypred_scale)
ypred

In [ ]:
pred_data=pd.DataFrame(ypred,columns=['SalePrice'])
target_pred=pd.concat([Id_pred,pred_data],axis=1)
target_pred.head()

In [ ]:
target_pred

In [ ]:
target_pred.to_csv("submission.csv", index=False)
print("Submission was successfully saved!")